In [1]:
# Auto-reload modules when they change
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

# Add src to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

print(f"Project root: {project_root}")
print(f"Python path: {sys.path[0]}")

Project root: c:\Users\H244746\Documents\reit-risk-summarizer
Python path: c:\Users\H244746\Documents\reit-risk-summarizer\src


## 1. Load Test Data

Load the AMT risk factors we extracted earlier.

In [3]:
# Load AMT risk factors from fixtures
risk_factors_path = project_root / 'tests' / 'fixtures' / 'sample_AMT_risk_factors.txt'

with open(risk_factors_path, 'r', encoding='utf-8') as f:
    amt_risk_text = f.read()

print(f"Loaded risk factors: {len(amt_risk_text):,} characters")
print(f"\nFirst 500 chars:\n{amt_risk_text[:500]}...")

Loaded risk factors: 119,552 characters

First 500 chars:
AMERICAN TOWER CORPORATION
TABLE OF CONTENTS—(Continued)
FORM 10-K ANNUAL REPORT
FISCAL YEAR ENDED DECEMBER 31, 2024  PageITEM 9A.Controls and Procedures59Disclosure Controls and Procedures59Management’s Annual Report on Internal Control over Financial Reporting60Changes in Internal Control over Financial Reporting60Report of Independent Registered Public Accounting Firm61ITEM 9B.Other Information62ITEM 9C.Disclosure Regarding Foreign Jurisdictions That Prevent Inspections62PART IIIITEM 10.Direc...


## 2. Set Up Environment

Configure API keys (make sure you have a `.env` file with your keys).

In [4]:
from dotenv import load_dotenv
import os
from reit_risk_summarizer.services.llm.summarizer import create_summarizer

# Load environment variables
load_dotenv(project_root / '.env')

# Check API keys are loaded and not placeholders
def is_valid_key(key):
    """Check if key exists and is not a placeholder."""
    if not key:
        return False
    placeholders = ['gsk-your-groq-key-here', 'your-key-here']
    return key not in placeholders

groq_key = os.getenv('GROQ_API_KEY')

has_groq = is_valid_key(groq_key)

print(f"Groq API Key configured: {has_groq}")

if not has_groq:
    print("\n⚠️ No Groq API key found! Add to .env file:")
    print("   GROQ_API_KEY=gsk_... (FREE at https://console.groq.com)")

Groq API Key configured: True


## 1. Test Groq Summarizer (FREE - Llama 3.3 70B)

Groq provides ultra-fast inference for open-source models with a generous free tier.

In [5]:
if has_groq:
    # Create Groq summarizer (DEFAULT model for this project)
    groq_summarizer = create_summarizer(
        provider="groq",
        model="llama-3.3-70b-versatile",  # FREE tier
        temperature=0.0,
        prompt_version="v1.0"
    )
    
    print(f"Created summarizer: {groq_summarizer.__class__.__name__}")
    print(f"Model: {groq_summarizer.model}")
    print(f"Temperature: {groq_summarizer.temperature}")
    print(f"Prompt version: {groq_summarizer.prompt_version}")
    print("💰 Cost: FREE (14,400 requests/day)")
else:
    print("⚠️ Skipping Groq test - no API key")
    print("   Get FREE API key at: https://console.groq.com")

Created summarizer: GroqRiskSummarizer
Model: llama-3.3-70b-versatile
Temperature: 0.0
Prompt version: v1.0
💰 Cost: FREE (14,400 requests/day)


In [7]:
if has_groq:
    print("🚀 Calling Groq API (ultra-fast inference - should be <5 seconds!)...\n")
    
    groq_result = groq_summarizer.summarize(
        risk_text=amt_risk_text,
        ticker="AMT",
        company_name="American Tower Corporation"
    )
    
    print("✅ Groq Summarization Complete\n")
    print(f"Model: {groq_result.model}")
    print(f"Prompt Version: {groq_result.prompt_version}")
    print(f"\nTop 5 Risks:\n")
    
    for i, risk in enumerate(groq_result.risks, 1):
        print(f"{i}. {risk}\n")

🚀 Calling Groq API (ultra-fast inference - should be <5 seconds!)...



Groq API error for AMT: Connection error.


LLMSummarizationError: Groq summarization failed: Connection error.

## 2. Test Hugging Face Summarizer (LOCAL - No API Required)

Hugging Face models run locally on your machine - no internet or API key needed!

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [7]:
auth_token = os.getenv('HF_TOKEN')

In [8]:
# Test HF_TOKEN authentication
# This checks if your token works without downloading the full model
print("Testing HF_TOKEN authentication...\n")

if auth_token:
    print(f"✅ HF_TOKEN found: {auth_token[:10]}...")
    
    try:
        from huggingface_hub import HfApi
        api = HfApi()
        
        # Test with a gated model (Llama 3.2) - just checks access, doesn't download
        model_id = "meta-llama/Llama-3.2-1B-Instruct"
        print(f"\nTesting access to gated model: {model_id}")
        
        # This will fail if token is invalid or you haven't accepted the license
        model_info = api.model_info(model_id, token=auth_token)
        
        print(f"✅ Token is valid!")
        print(f"✅ Model info retrieved: {model_info.modelId}")
        print(f"   Pipeline: {model_info.pipeline_tag}")
        print(f"   Downloads: {model_info.downloads:,}")
        
        # Check if you have access
        if model_info.gated:
            print(f"\n⚠️ Model is gated. Status: {model_info.gated}")
            print("   You may need to:")
            print("   1. Visit https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct")
            print("   2. Accept the license agreement")
            print("   3. Wait for approval (usually instant for Llama 3.2)")
        else:
            print("✅ You have access to this model!")
            
    except ImportError:
        print("⚠️ huggingface_hub not installed")
        print("   Run: uv pip install huggingface_hub")
    except Exception as e:
        print(f"❌ Token validation failed: {e}")
        print("\nPossible issues:")
        print("1. Token is invalid (regenerate at https://huggingface.co/settings/tokens)")
        print("2. Haven't accepted Llama license at https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct")
        print("3. Firewall blocking huggingface.co API")
else:
    print("❌ No HF_TOKEN found in environment")
    print("   Add to .env file: HF_TOKEN=hf_your_token_here")

Testing HF_TOKEN authentication...

✅ HF_TOKEN found: hf_nEMSMOp...

Testing access to gated model: meta-llama/Llama-3.2-1B-Instruct
✅ Token is valid!
✅ Model info retrieved: meta-llama/Llama-3.2-1B-Instruct
   Pipeline: text-generation
   Downloads: 3,473,341

⚠️ Model is gated. Status: manual
   You may need to:
   1. Visit https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct
   2. Accept the license agreement
   3. Wait for approval (usually instant for Llama 3.2)
✅ Token is valid!
✅ Model info retrieved: meta-llama/Llama-3.2-1B-Instruct
   Pipeline: text-generation
   Downloads: 3,473,341

⚠️ Model is gated. Status: manual
   You may need to:
   1. Visit https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct
   2. Accept the license agreement
   3. Wait for approval (usually instant for Llama 3.2)


In [42]:
# Test Hugging Face - runs locally, no API needed
# NOTE: First download requires internet access to Hugging Face
# If behind a firewall, this may not work

has_huggingface = False

# Try 3B model - better quality than 1B (still fast enough on CPU)
try:
    hf_summarizer = create_summarizer(
        provider="huggingface",
        model="meta-llama/Llama-3.2-3B-Instruct",  # Larger model for better results
        temperature=0.0,
        prompt_version="v1.0"
    )
    
    print(f"Created summarizer: {hf_summarizer.__class__.__name__}")
    print(f"Model: {hf_summarizer.model}")
    print("💻 Runs locally - no internet required after download!")
    print("⚠️ First run will download model (~6GB)")
    print("⏱️ Will take 2-3 minutes on CPU (vs 1 min for 1B model)")
    
    has_huggingface = True
except Exception as e:
    print(f"⚠️ Hugging Face not available: {e}")
    has_huggingface = False


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

c:\Users\H244746\Documents\reit-risk-summarizer\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\H244746\.cache\huggingface\hub\models--meta-llama--Llama-3.2-3B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


Created summarizer: HuggingFaceRiskSummarizer
Model: meta-llama/Llama-3.2-3B-Instruct
💻 Runs locally - no internet required after download!
⚠️ First run will download model (~6GB)
⏱️ Will take 2-3 minutes on CPU (vs 1 min for 1B model)


In [44]:
print(f"max_tokens setting: {hf_summarizer.max_tokens}")

max_tokens setting: 300


In [45]:
if has_huggingface:
    print("🚀 Running local Hugging Face model (may take 30-60 seconds)...\n")
    
    hf_result = hf_summarizer.summarize(
        risk_text=amt_risk_text,
        ticker="AMT",
        company_name="American Tower Corporation"
    )
    
    print("✅ Hugging Face Summarization Complete\n")
    print(f"Model: {hf_result.model}")
    print(f"Prompt Version: {hf_result.prompt_version}")
    print(f"\nTop 5 Risks:\n")
    
    for i, risk in enumerate(hf_result.risks, 1):
        print(f"{i}. {risk}\n")

🚀 Running local Hugging Face model (may take 30-60 seconds)...



: 

In [1]:
import psutil
mem = psutil.virtual_memory()
print(f"Total RAM: {mem.total / (1024**3):.1f} GB")
print(f"Available: {mem.available / (1024**3):.1f} GB")
print(f"Used: {mem.used / (1024**3):.1f} GB ({mem.percent}%)")

Total RAM: 31.7 GB
Available: 17.7 GB
Used: 14.0 GB (44.2%)


In [35]:
# Debug: Print the raw response to see what the model generated
if has_huggingface and 'hf_result' in locals():
    print("Raw model response:")
    print("=" * 80)
    print(hf_result.raw_response)
    print("=" * 80)

Raw model response:
system

Cutting Knowledge Date: December 2023
Today Date: 10 Dec 2025

You are a financial analyst specializing in Real Estate Investment Trust (REIT) analysis. Your expertise is in reading SEC 10-K filings and identifying the most material risks that could impact investor returns.

Your task is to analyze the Risk Factors section (Item 1A) and identify the TOP 5 MOST MATERIAL risks.

Key principles:
1. **Materiality**: Focus on risks with the highest potential financial impact
2. **Specificity**: Prioritize risks specific to this REIT's business model, not generic industry risks
3. **Actionability**: Risks should be concrete and measurable, not vague statements
4. **Investor Focus**: Consider what matters most to REIT investors (dividend stability, property values, occupancy rates)

Output Format:
- Provide exactly 5 risks, numbered 1-5
- Rank them from most to least material
- Each risk should be 2-3 sentences
- Be clear and conciseuser

Analyze the following Risk

In [29]:
print(amt_risk_text)

AMERICAN TOWER CORPORATION
TABLE OF CONTENTS—(Continued)
FORM 10-K ANNUAL REPORT
FISCAL YEAR ENDED DECEMBER 31, 2024  PageITEM 9A.Controls and Procedures59Disclosure Controls and Procedures59Management’s Annual Report on Internal Control over Financial Reporting60Changes in Internal Control over Financial Reporting60Report of Independent Registered Public Accounting Firm61ITEM 9B.Other Information62ITEM 9C.Disclosure Regarding Foreign Jurisdictions That Prevent Inspections62PART IIIITEM 10.Directors, Executive Officers and Corporate Governance63ITEM 11.Executive Compensation65ITEM 12.Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters65ITEM 13.Certain Relationships and Related Transactions, and Director Independence65ITEM 14.Principal Accounting Fees and Services65PART IVITEM 15.Exhibits, Financial Statement Schedules66Index to Exhibits66ITEM 16.Form 10-K Summary76Signatures77Index to Consolidated Financial StatementsF-1
SPECIAL NOTE REGARDING

## 6. Compare Results

Let's compare the outputs from all available models.

In [ ]:
# Build list of available results
results = []
if has_groq:
    results.append(("Groq (FREE)", groq_result))
if has_huggingface:
    results.append(("HuggingFace (LOCAL)", hf_result))

if len(results) >= 2:
    print("=" * 80)
    print(f"COMPARING {len(results)} MODELS")
    print("=" * 80)
    
    for i in range(5):
        print(f"\n{'='*80}")
        print(f"RISK #{i+1}")
        print(f"{'='*80}\n")
        
        for label, result in results:
            print(f"🤖 {label} ({result.model}):")
            print(f"   {result.risks[i]}\n")
elif len(results) == 1:
    label, result = results[0]
    print(f"Only {label} results available.")
else:
    print("⚠️ No results to compare")
    print("\nRecommended: Start with Groq (FREE)")
    print("Get API key at: https://console.groq.com")

## 7. Validate RiskSummary Structure

Check that the output follows the expected schema.

In [ ]:
from reit_risk_summarizer.services.llm.summarizer import RiskSummary

def validate_risk_summary(summary: RiskSummary, label: str):
    """Validate RiskSummary structure."""
    print(f"\n{'='*60}")
    print(f"Validating {label}")
    print(f"{'='*60}")
    
    checks = [
        ("Has exactly 5 risks", len(summary.risks) == 5),
        ("All risks are non-empty strings", all(isinstance(r, str) and r.strip() for r in summary.risks)),
        ("Has ticker", bool(summary.ticker)),
        ("Has company name", bool(summary.company_name)),
        ("Has model", bool(summary.model)),
        ("Has prompt version", bool(summary.prompt_version)),
        ("Has raw response", bool(summary.raw_response)),
    ]
    
    all_passed = True
    for check_name, passed in checks:
        status = "✅" if passed else "❌"
        print(f"{status} {check_name}")
        if not passed:
            all_passed = False
    
    print(f"\n{'✅ All checks passed!' if all_passed else '❌ Some checks failed'}\n")
    return all_passed

# Validate all available results
for label, result in results:
    validate_risk_summary(result, f"{label} Result")

## 8. Test with Truncated Risk Text

Test with a smaller sample to verify it works with less data.

In [ ]:
# Use first available summarizer (prefer HuggingFace for local/offline)
if has_huggingface:
    test_summarizer = hf_summarizer
    test_label = "HuggingFace (LOCAL)"
elif has_groq:
    test_summarizer = groq_summarizer
    test_label = "Groq"
elif has_openai:
    test_summarizer = openai_summarizer
    test_label = "OpenAI"
elif has_anthropic:
    test_summarizer = anthropic_summarizer
    test_label = "Anthropic"
else:
    test_summarizer = None
    test_label = None

if test_summarizer:
    # Use only first 10,000 characters
    truncated_text = amt_risk_text[:10000]
    
    print(f"Testing {test_label} with truncated text ({len(truncated_text):,} chars)...\n")
    
    truncated_result = test_summarizer.summarize(
        risk_text=truncated_text,
        ticker="AMT",
        company_name="American Tower Corporation"
    )
    
    print("✅ Truncated test successful\n")
    print("Top 3 risks from truncated text:\n")
    for i in range(3):
        print(f"{i+1}. {truncated_result.risks[i]}\n")
else:
    print("⚠️ No API keys available for truncation test")

## 9. Test Error Handling

Test what happens with invalid input.

In [ ]:
if test_summarizer:
    from reit_risk_summarizer.exceptions import LLMSummarizationError
    
    # Test with very short text (might not produce 5 risks)
    try:
        short_text = "There are some risks."
        print(f"Testing {test_label} with very short text...\n")
        
        short_result = test_summarizer.summarize(
            risk_text=short_text,
            ticker="TEST",
            company_name="Test Company"
        )
        print("✅ Somehow worked (LLM might have hallucinated risks)")
        
    except LLMSummarizationError as e:
        print(f"✅ Properly caught error: {e}")
    except Exception as e:
        print(f"⚠️ Unexpected error type: {type(e).__name__}: {e}")
else:
    print("⚠️ No API keys available for error handling test")

## Summary

The summarizer is working! Key takeaways:
- ✅ **Two LLM providers**: Groq (Llama 3.3 70B - FREE!), **HuggingFace (LOCAL - Llama 3.2-1B)**
- ✅ **HuggingFace/Llama 3.2-1B is the default** - runs 100% locally, no API/internet needed
- ✅ Groq offers 14,400 free requests/day with ultra-fast inference (<5 seconds)
- ✅ HuggingFace bypasses firewalls - downloads model once (~2.5GB), then works completely offline
- ✅ Structured output with RiskSummary dataclass
- ✅ Prompt versioning for evaluation tracking
- ✅ Error handling in place
- ✅ Factory pattern for easy provider switching

**Next steps:**
1. Create unit tests for summarizer
2. Build orchestrator to tie fetcher → extractor → summarizer together
3. Integrate with evaluation framework
4. Add API endpoints